# Example of CVaR based optimal portfolio backtesting.

We consider a Sharpe (maximization) optimization strategy, i.e. rtype="Sharpe" (default value).

Similar results can be obtained for other optimization strategies by setting rtype to "Risk", "MinRisk" and "InvNrisk".

In [37]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Collect MkT Data 
1. setup start and (sdate) end date (edate) of the historical MkT Data.
2. define the list of symbols under consideration
3. define the local directory where to save the MkT Data for later use (optional)
4. call readMkT with appropriate setup

In [26]:
sdate = pd.Timestamp("2012-01-01").normalize()
edate = pd.Timestamp.today().normalize()
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

mktdir = "./MkTdata"

# force=True read from alphavantage server
# force=False read from local directory if data exists
rprice = az.readMkT(symb, dstart = sdate, dend = edate, 
                    dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


### Define the CVaR dispersion measure parameters
We use a equally weighted compounding CVaR measure for 3 confidence levels. 

In [27]:
alpha = [0.99, 0.975, 0.95]
coef = np.ones(len(alpha))
coef = coef / coef.sum()

### Compute the C-Sharpe optimal portfolio backtesting
Fist we construct the class Port_CVaR and then compute the portfolio time series.

In [28]:
p5 = az.Port_CVaR(rprice, symb=symb, sdate=sdate, edate=edate, hlenght=3.26) 
port5 = p5.get_port(mu=0., alpha=alpha, coef=coef)   

### Two graphical representations:
1. Portfolio daily time series with technical indicators (EMA for 30 and 200 days).
2. Relative performance of the portfolio and its components (the curves are rescaled to 1 at inception).

The fancy flag is set to True in order to use the plotly library for time series graphical representations.
(If fancy=False then the matplotlib is used - with no zoom facilities)

In [29]:
v1 = p5.port_view(fancy=True)
v2 = p5.port_view_all(fancy=True)

### Portfolio performance where:
RR - average anual rate of return
DD - maximum drawdown rate
DD_date - maximum drawdown date
DD_start - maximum drawdown start date
DD_end - maximum drawdown end date

Note: we have used the option fancy=True in order to report the rates in percents.

In [30]:
p5.port_perf(fancy=True)

,RR,DD,DD_date,DD_start,DD_end
symbol,,,,,
CVaR_Port,10.61,-19.10,2020-03-20,2020-02-19,2020-05-20
GLD,1.26,-42.11,2015-12-17,2012-10-04,2020-07-22
PSJ,22.53,-30.69,2020-03-16,2020-02-19,2020-05-22
TLT,1.56,-22.96,2013-12-27,2012-07-25,2015-01-14
VGT,21.01,-32.11,2020-03-23,2020-02-19,2020-06-10
XLV,14.24,-28.75,2020-03-23,2020-01-22,2020-07-17


### Portfolio drawdowns.
By default the first 5 largest drawdown are reported (it can be change by setting top to a new value)
DD - drawdown rate
Date - drawdown date
Start - drawdown start date
End - drawdown end date
A value of NaN for End idicates that the drawdown is still in progress and deeper levels are still possible.

Note: we have used the option fancy=True in order to report DD in percents.

In [31]:
p5.port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-19.10,2020-03-20,2020-02-19,2020-05-20
2,-14.92,2015-09-28,2015-08-05,2016-07-27
3,-14.42,2018-12-24,2018-08-31,2019-02-14
4,-12.28,2021-03-08,2020-08-06,NaN
5,-9.34,2016-12-01,2016-09-28,2017-04-27


### Annual rates of returns 
The rates of retuns are reported for each calendar year.

Note: we have used the option fancy=True in order to report RR in percents.

In [32]:
p5.port_annual_returns(fancy=True)

,RR
year,
2015,-7.31%
2016,2.59%
2017,23.81%
2018,5.09%
2019,24.57%
2020,17.36%
2021,-1.54%


### Monthly rates of returns 
The rates of retuns are reported for each calendar month.

Note: we have used the option fancy=True in order to report RR in percents and in visualy conveninet format.

In [33]:
p5.port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-1.44%,2.23%,2.96%,5.87%,2.89%,-2.72%
2,nan%,1.57%,3.41%,-0.36%,3.28%,-3.96%,-4.34%
3,nan%,1.54%,1.20%,2.32%,-0.20%,-4.31%,-2.07%
4,nan%,-0.66%,1.82%,0.65%,2.48%,8.22%,2.26%
5,nan%,2.30%,3.20%,4.54%,-1.57%,3.29%,2.82%
6,-1.76%,2.95%,-3.15%,-2.62%,5.94%,0.84%,-1.13%
7,2.22%,3.31%,2.88%,0.45%,3.84%,8.06%,nan%
8,-7.34%,-0.77%,2.61%,5.72%,-0.70%,-0.16%,nan%
9,-4.33%,0.64%,1.73%,-0.89%,-2.69%,-3.31%,nan%


### Each period weights
Droll - rolling date (new position are aquired at the end of day)
Dfix - fixing date (the weights are computed based on this day close prices)

In [34]:
p5.get_weights()

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,0.000000,0.000000,0.086195,0.000000,0.913805
1,2015-09-25,2015-09-24,2012-06-22,0.000000,0.220248,0.362704,0.000000,0.417048
2,2015-12-28,2015-12-24,2012-09-24,0.000000,0.365146,0.375570,0.062380,0.196904
3,2016-03-28,2016-03-24,2012-12-24,0.000000,0.047148,0.416359,0.163060,0.373433
4,2016-06-27,2016-06-24,2013-03-22,0.000000,0.143296,0.432479,0.104178,0.320047
5,2016-09-27,2016-09-26,2013-06-26,0.000000,0.195354,0.458306,0.066922,0.279418
6,2016-12-27,2016-12-23,2013-09-23,0.000000,0.000000,0.403213,0.596787,0.000000
7,2017-03-28,2017-03-27,2013-12-27,0.000000,0.000000,0.394851,0.605149,0.000000
8,2017-06-27,2017-06-26,2014-03-26,0.000000,0.000000,0.393625,0.606375,0.000000
9,2017-09-26,2017-09-25,2014-06-25,0.000000,0.069088,0.390749,0.540163,0.000000


### Number of shares for each basket components on each rolling period
Droll is the rolling date. The new positions are assumed to be aquired at the close price.

In [35]:
p5.get_nshares()

,GLD,PSJ,TLT,VGT,XLV
Droll,,,,,
2015-06-25,0,0,74,0,1207
2015-09-25,0,477,265,0,544
2015-12-28,0,801,289,54,255
2016-03-28,0,108,299,141,516
2016-06-27,0,323,307,96,445
2016-09-27,0,416,341,58,397
2016-12-27,0,0,323,458,0
2017-03-28,0,0,329,456,0
2017-06-27,0,0,327,447,0


### Accounting information
For each investment period indicated by the roling day, Droll we have
- for each basket component the number of shares
- cash_invst - the amount of cash invested in shares
- cash_roll - residual cash position during the rolling period
- cash_divd - cash dividend accumulate in the previouse rolling period and availabel for investing in Droll

Note: The total Capital in the Droll is cash_invst + cash_roll. 

In [36]:
p5.get_account(fancy=True)  

,GLD,PSJ,TLT,VGT,XLV,cash_invst,cash_roll,cash_divd
Droll,,,,,,,,
2015-06-25,0,0,74,0,1207,100476.66,-476.66,0.00
2015-09-25,0,477,265,0,544,88205.06,1409.40,373.00
2015-12-28,0,801,289,54,255,93917.46,-26.62,438.57
2016-03-28,0,108,299,141,516,92931.94,148.66,223.77
2016-06-27,0,323,307,96,445,96335.79,-3.69,435.09
2016-09-27,0,416,341,58,397,103344.93,-782.30,398.24
2016-12-27,0,0,323,458,0,94703.23,-127.25,509.94
2017-03-28,0,0,329,456,0,101344.30,-255.01,325.53
2017-06-27,0,0,327,447,0,105025.14,1532.06,254.12
